# Criação do CSV do Consumption Profile Parcial
## Versão mais leve com bases parciais para executar em sala

# Food Intake 

Ativando uma conexão de banco de dados em memória usando o SGBD H2:

In [1]:
%defaultDatasource jdbc:h2:mem:db

## Conjunto de tabelas para consultas para análise e construir profile

In [2]:
DROP TABLE IF EXISTS Intake;

CREATE TABLE Intake (
  SeqN INTEGER NOT NULL,
  DayCode TINYINT NOT NULL,
  DraBF TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Intake DECIMAL(13,7),
  Intake_BW DECIMAL(13,10),
  PRIMARY KEY(SeqN, DayCode, FCID_Code, Cooked_Status, Food_Form, Cooking_Method)
) AS SELECT
  SEQN, DAYCODE, DRABF, FCID_Code, Cooked_Status, Food_Form, Cooking_Method, Intake,Intake_BW
FROM CSVREAD('../../../data/food-intake/consumption/Commodity_CSFFM_Intake_0510-cropped.csv');

In [3]:
DROP TABLE IF EXISTS Recipes;

CREATE TABLE Recipes (
  Food_Code VARCHAR(8),
  Mod_Code VARCHAR(8),
  Ingredient_Num TINYINT,
  FCID_Code VARCHAR(10),
  Cooked_Status TINYINT,
  Food_Form TINYINT,
  Cooking_Method TINYINT,
  Commodity_Weight DECIMAL(5, 2),
  CSFII_9498_IND TINYINT,
  WWEIA_9904_IND TINYINT,
  WWEIA_0510_IND TINYINT,
  PRIMARY KEY(Food_Code, Mod_Code, Ingredient_Num)
) AS SELECT
  Food_Code, Mod_Code, Ingredient_Num, FCID_Code, Cooked_Status, Food_Form, Cooking_Method,
  Commodity_Weight, CSFII_9498_IND, WWEIA_9904_IND, WWEIA_0510_IND
FROM CSVREAD('../../../data/food-intake/recipes/Recipes_WWEIA_FCID_0510.csv');

In [4]:
DROP TABLE IF EXISTS FCID_Description;

CREATE TABLE FCID_Description (
  CGN VARCHAR(2),
  CG_Subgroup VARCHAR(6),
  FCID_Code VARCHAR(10),
  FCID_Desc VARCHAR(55),
  PRIMARY KEY (FCID_Code)
) AS SELECT
  cgn, CG_Subgroup, FCID_Code, FCID_Desc
FROM CSVREAD('../../../data/food-intake/basics/FCID_Code_Description.csv');

# Apresenção e Análises Básicas das Tabelas

## Ingredientes (Produtos Alimentares de Base)

Cada tupla é um produto alimentar de base usado em receitas.

| Campo | Descrição | Chave para a Tabela |
| --- | --- | --- |
| CGN | código do grupo de cultura (agrícola) | associado à tabela de Grupos e Subgrupos de Cultura |
| CG_Subgroup | código do subgrupo de cultura (agrícola) | associado à tabela de Grupos e Subgrupos de Cultura |
| FCID_Code | código do ingrediente | |
| FCID_Desc | descrição do ingrediente | |

In [5]:
SELECT * FROM FCID_Description LIMIT 10;

## Consumo

Cada tupla é um produto alimentar (ingrediente) ingerido por uma pessoa em um dia. Todas as tuplas com mesmo `SEQN` são ingestões da mesma pessoa no dia 1 ou 2 (`DAYCODE`).

| Campo | Descrição | Chave para a Tabela ou Valor |
| --- | --- | --- |
| SEQN  | id do entrevistado | |
| DRABF | indica se o entrevistado (bebê) foi amamentado em qualquer um dos dois dias de retorno | 1 = sim; 2 = não |
| DAYCODE | Dia do diário alimentar. | 1 = dia 1; 2= dia 2 |
| FCID_Code | código do produto alimentar (usado como ingrediente) | Ingredientes (Produtos Alimentares) |
| Cooked_Status | estado e cozimento original do ingrediente | Estado de Cozimento do Produto |
| Food_Form | forma alimentar | Forma do Produto Alimentar |
| Cooking_Method | método de cozimento do ingrediente na receita | Método de Cozimento do Produto na Receita |
| Intake | ingestão. em gramas | |
| Intake_BW | ingestão. em gramas por quilograma de peso corporal (g/kg pc) | |

In [6]:
SELECT * FROM Intake LIMIT 10;

## Receitas - Composição

Cada tupla é um item que é parte de uma receita. Cada receita - todas as tuplas com o mesmo `Food_code`.

| Campo | Descrição | Chave para a Tabela |
| --- | --- | --- |
| Food_Code | código da receita | Alimentos feitos por Receitas |
| Mod_Code | código de receita modificada (0 = sem modificação) | |
| Ingredient_Num | número do ingrediente dentro da receita | |
| FCID_Code | código do ingrediente | Ingredientes (Produtos Alimentares) |
| Cooked_Status | estado e cozimento original do ingrediente | Estado de Cozimento do Produto |
| Food_Form | forma alimentar | Forma do Produto Alimentar |
| Cooking_Method | método de cozimento do ingrediente na receita | Método de Cozimento do Produto na Receita |
| Commodity_Weight | participação (em percentual) do ingrediente no peso da receita | |

In [7]:
SELECT * FROM Recipes LIMIT 10;

In [8]:
SELECT * FROM Recipes WHERE Food_Code = '27111300' AND Mod_Code = 0;

In [9]:
SELECT SUM(Commodity_Weight) FROM Recipes WHERE Food_Code = '27111300' AND Mod_Code = 0;

99.79

In [10]:
SELECT MAX(SeqN) FROM Intake;

32776

In [11]:
SELECT COUNT(DISTINCT Food_Code) FROM Recipes;

7154

In [12]:
DROP VIEW IF EXISTS Participation_Profile;

CREATE VIEW Participation_Profile AS
SELECT R.FCID_Code, COUNT(DISTINCT (R.Food_Code, R.Mod_Code)) Participation
FROM Recipes R
GROUP BY R.FCID_Code;

SELECT * FROM Participation_Profile LIMIT 10;

In [13]:
SELECT COUNT(*) FROM Intake;

100000

In [14]:
SELECT COUNT(DISTINCT SEQN) FROM Intake;

1489

In [15]:
DROP VIEW IF EXISTS Commodity_Profile;

CREATE VIEW Commodity_Profile
AS SELECT I.FCID_Code, F.FCID_Desc, F.CGN, F.CG_Subgroup, COUNT(DISTINCT I.SeqN) Popularity,
          SUM(I.Intake) Intake_Sum, AVG(I.Intake) Intake_AVG,
          AVG(I.Intake_BW) Intake_BW_AVG, P.Participation Recipes
FROM Intake I, FCID_Description F, Participation_Profile P
WHERE I.FCID_Code = F.FCID_Code AND I.FCID_Code = P.FCID_Code
GROUP BY I.FCID_Code;

In [16]:
SELECT SUM(Intake_Sum) FROM Commodity_Profile;

3224434.0950660

In [17]:
CALL CSVWRITE('../../../data/food-intake/profile/commodity-profile-partial.csv',
              'SELECT * FROM Commodity_Profile');

368